In [28]:
# Carica il file csv e coonverto colonna dei prezzi in int
import pandas as pd
import re
import itertools

df=pd.read_csv("dati_roma/roma_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df.columns

Index(['nome_hotel', 'prezzo', 'stanza', 'città', 'data', 'punteggio',
       'numero_recensioni', 'distanza_centro', 'genius', 'offerte',
       'colazione_inclusa', 'senza_pagamento_anticipato',
       'cancellazione_gratuita', 'os', 'username', 'user_agent_type',
       'TIMESTAMP'],
      dtype='object')

In [29]:
#Prendo tutte le combinazioni di coppie di ricerche sulla stessa città
df_list = df['TIMESTAMP'].unique()
coppie_df = list(itertools.combinations(df_list, 2))
len(coppie_df)


45

In [30]:
# Creo un Dataframe per vedere differenze di prezzo tra diverse ricerche
list_dict_df = []
for i in coppie_df:
    orario1 = i[0]
    orario2 = i[1]
    df1 = df[df["TIMESTAMP"]==orario1]
    df2 = df[df["TIMESTAMP"]==orario2]
    df1 = df1.drop_duplicates()
    df2 = df2.drop_duplicates()
    if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data'], how='inner')
    else:
        merged_df = pd.merge(df1, df2, on=['nome_hotel','data'], how='inner')
    initial_len = len(merged_df)
    merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
    # Elimino hotel con stesso nome 
    righe_droppate = len(merged_df)
    merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)
    righe_droppate -= len(merged_df)
    merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"]).abs()
    if merged_df['prezzo_x'].sum() < merged_df['prezzo_y'].sum():
        prezzi_inferiori = "Dataframe 1"
    else: prezzi_inferiori = "Dataframe 2"
    dict_df = {}
    dict_df ['coppia_ricerche'] = orario1+" *** "+orario2
    dict_df ['num_differenze_prezzo'] = len(merged_df)
    dict_df ['media'] = merged_df['differenza_prezzo'].mean()
    dict_df ['mediana'] = merged_df['differenza_prezzo'].median()
    dict_df['%_prezzi_diversi'] = len(merged_df)/initial_len
    dict_df['tipo_ricerche'] = df1['user_agent_type'].unique()[0] + "/"+ df2['user_agent_type'].unique()[0]
    dict_df['prezzi_inferiori'] = prezzi_inferiori
    dict_df['righe_duplicate'] = righe_droppate
    list_dict_df.append(dict_df)

df_differenze = pd.DataFrame(list_dict_df)   
df_differenze.sort_values(by=['num_differenze_prezzo','media'],ascending=False)


,coppia_ricerche,num_differenze_prezzo,media,mediana,%_prezzi_diversi,tipo_ricerche,prezzi_inferiori,righe_duplicate
41,03-03-2024---15:22 *** 03-03-2024---16:44,552,12.483696,10.0,0.751020,mobile/desktop,Dataframe 1,3
20,03-03-2024---14:16 *** 03-03-2024---15:22,550,15.507273,13.0,0.754458,desktop/mobile,Dataframe 2,3
35,03-03-2024---15:12 *** 03-03-2024---15:22,536,13.106343,11.0,0.751753,desktop/mobile,Dataframe 2,3
13,03-03-2024---14:02 *** 03-03-2024---15:22,523,12.405354,10.0,0.669654,mobile/mobile,Dataframe 2,0
39,03-03-2024---15:22 *** 03-03-2024---15:37,508,10.978346,10.0,0.648787,mobile/mobile,Dataframe 1,0
31,03-03-2024---15:01 *** 03-03-2024---15:22,506,12.294466,10.0,0.743025,desktop/mobile,Dataframe 2,3
40,03-03-2024---15:22 *** 03-03-2024---16:00,504,10.980159,10.0,0.647815,mobile/mobile,Dataframe 1,0
26,03-03-2024---14:31 *** 03-03-2024---15:22,501,13.215569,11.0,0.763720,desktop/mobile,Dataframe 2,3
5,03-03-2024---13:49 *** 03-03-2024---15:22,442,9.061086,8.0,0.627841,mobile/mobile,Dataframe 2,0
16,03-03-2024---14:02 *** 03-03-2024---16:44,362,15.560773,13.0,0.430952,mobile/desktop,Dataframe 1,2


In [ ]:
# Elimino una o più ricerche dal dataframe
new_df = df_differenze.drop(df_differenze[df_differenze['coppia_ricerche'].str.contains("20:42")].index) 
new_df.sort_values(by=['num_differenze_prezzo','media'],ascending=False)

In [ ]:
#Visualizza dataframe mobile/mobile o desktop/desktop
df_tipo_uguale = df_differenze[(df_differenze['tipo_ricerche']=="mobile/mobile") | (df_differenze['tipo_ricerche']=="desktop/desktop")]
df_tipo_uguale.sort_values(by="num_differenze_prezzo",ascending= False)

In [ ]:
# Visualizza il numero di ricerche mobile e desktop
mobile = 0
desktop = 0
for i in df_list:
    df1 = df[df["TIMESTAMP"]==i]
    stringa = df1['user_agent_type'].unique()[0]
    if "mob" in stringa:
        mobile +=1
    else: desktop+=1
    print(df1['user_agent_type'].unique())

print("Mobile:", mobile,"Desktop:",desktop)